## 준비하기
1. 아래 예제를 실행하기 위해서는 사전에 ./setup.sh 파일을 실행하거나 ./start_notebook.sh 로 노트북을 구동하여 의존성 라이브러리들을 미리 설치해두셔야 합니다.
2. qdrant서버가 떠 있어야 정상 동작합니다. ./start_qdrant.sh 를 실행하여 qdrant 서버를 띄워주세요.

## 목적:
- Qdrant 데이터베이스의 스냅샷을 생성하고, 필요할 때 복구 후 삭제

## 왜 필요한가?
- 데이터베이스를 안전하게 백업하고 필요할 때 빠르게 복원 가능
- 시스템 장애 발생 시 데이터 손실을 방지

## 주요 개념:
- **스냅샷(Snapshot)**: 데이터베이스의 특정 시점 상태를 저장하는 기능
- **백업 및 복구**: 예상치 못한 데이터 손실을 방지하고 시스템을 복구하는 과정

In [1]:
# 라이브러리 로딩 (아래 부분에서 오류가 발생하면 pip -r ./requirements.txt 로 의존성을 설치해주세요)
from qdrant_client import QdrantClient
from qdrant_client import models

In [2]:
# Qdrant 클라이언트 설정
qdrant_client = QdrantClient(host="localhost", port=6333)

In [3]:
collection_name = "ex11_sample_vectors"

# 0. 컬렉션 생성 (이미 존재하면 에러 발생 방지)
existing_collections = [c.name for c in qdrant_client.get_collections().collections]
if collection_name not in existing_collections:
    qdrant_client.create_collection(
        collection_name=collection_name,
        vectors_config=models.VectorParams(size=128, distance=models.Distance.COSINE),
    )
    print(f"컬렉션 `{collection_name}` 생성 완료!")
else:
    print(f"컬렉션 `{collection_name}` 이미 존재함.")

컬렉션 `ex11_sample_vectors` 생성 완료!


In [4]:
# 1. 스냅샷 생성
snapshot_info = qdrant_client.create_snapshot(collection_name=collection_name)
snapshot_name = snapshot_info.name
print(f"스냅샷 생성 완료: {snapshot_name}")

스냅샷 생성 완료: ex11_sample_vectors-4246091874186851-2025-02-16-11-13-52.snapshot


In [5]:
# 2. 스냅샷 목록 조회
snapshots = qdrant_client.list_snapshots(collection_name=collection_name)
print("스냅샷 목록:")
for snapshot in snapshots:
    print(f"- {snapshot.name} (생성 시간: {snapshot.creation_time}, 크기: {snapshot.size} bytes)")

스냅샷 목록:
- ex11_sample_vectors-4246091874186851-2025-02-16-11-13-52.snapshot (생성 시간: 2025-02-16T11:13:52, 크기: 509440 bytes)


In [6]:
# 3. 스냅샷 복구
# 스냅샷의 실제 저장 경로 설정
snapshot_path = f"file:///qdrant/snapshots/{collection_name}/{snapshot_name}"

print(f"스냅샷 복구 시도: {snapshot_path}")
qdrant_client.recover_snapshot(collection_name=collection_name, location=snapshot_path)
print("스냅샷 복구 완료!")

스냅샷 복구 시도: file:///qdrant/snapshots/ex11_sample_vectors/ex11_sample_vectors-4246091874186851-2025-02-16-11-13-52.snapshot
스냅샷 복구 완료!


In [7]:
# 4. 스냅샷 삭제 (복구 후)
qdrant_client.delete_snapshot(collection_name=collection_name, snapshot_name=snapshot_name)
print(f"스냅샷 삭제 완료: {snapshot_name}")


스냅샷 삭제 완료: ex11_sample_vectors-4246091874186851-2025-02-16-11-13-52.snapshot
